# Applied Machine Learning Systems (MLS-1) - ELEC0134

# Final Assignment - Classification of Tumours

## Task A - Binary Classification using Logistic Regression

## Completed by Student Number - 18014580

This is a Jupyter Notebook submitted as part of the final assignment for the Applied Machine Learning Systems (MLS-1) coursework which involves tumor classification and identification.

This particular notebook tests the logistic regression method on Task A to find the accuracy of logistic regression for this particular task.

### Importing different packages

Initially, we must import the different packages needed for this task. The packages required to implement logistic regression for this binary task are pandas, sklearn, numpy and scipy.

In [1]:
import pandas as pd
import numpy as np
# import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from skimage import io
from skimage.io import imread, imshow

from keras.preprocessing import image
import matplotlib.pyplot as plt

import os
from os import listdir

from datetime import datetime

print("All imports carried out successfully")

All imports carried out successfully


### Loading the data from the data sets

In [2]:
#Loading the CSV file containing the labels
tumour_labels = pd.read_csv('./dataset/label.csv')
print(tumour_labels.shape) #outputs array with the number of features + feature number

(3000, 2)


### Loading the images

In [5]:
#example of loading one image
#image = imread('./dataset/image/IMAGE_0009.jpg')
#plt.imshow(image)

#image2 = imread('./dataset/image/IMAGE_0010.jpg')
#plt.figure()
#plt.imshow(image2)
#imarr = np.array(image2)
#plt.figure()
#plt.imshow(imarr)

images = []
#x=0
#t = time.process_time()

#dirname = './dataset/image'
#print(listdir(dirname))

#start = datetime.now()
#start = datetime.now().time()
#print('start: ')
#print(start)

for file in listdir('./dataset/image'):

    img = io.imread('./dataset/image/' + file)
    #plt.figure()
    #plt.imshow(img)
    
    #print(x)
    #x += 1
    img = np.array(img)
    images.append(img)

print(np.array(images).shape)
    
#end = datetime.now()
#end = datetime.now().time()
#print('end: ')
#print(end)
#elapsed = end - start
#print(elapsed)
#current = elapsed
    
#ETA = time.process_time() - t
#print(ETA)
    


(3000, 512, 512, 3)


### Splitting the data into training and testing data

In [7]:
X = np.array(images)
Y = tumour_labels['label']

xTrain, xTest, yTrain, yTest = train_test_split(X,Y)

print(xTrain.shape)
print(xTest.shape)

(2250, 512, 512, 3)
(750, 512, 512, 3)


### Feature extraction